## WIP: Demo notebook for plotly plots just to get feedback

In [1]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [2]:
import warnings

import arviz as az
import matplotlib.pyplot as plt
from pymc_extras.prior import Prior

from pymc_marketing.mmm import GeometricAdstock, LogisticSaturation
from pymc_marketing.mmm.multidimensional import (
    MMM,
    MultiDimensionalBudgetOptimizerWrapper,
)
from pymc_marketing.paths import data_dir
from pymc_marketing.special_priors import LogNormalPrior

warnings.filterwarnings("ignore", category=FutureWarning)

az.style.use("arviz-darkgrid")
plt.rcParams["figure.figsize"] = [12, 7]
plt.rcParams["figure.dpi"] = 100

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

import plotly.io as pio

pio.renderers.default = "notebook_connected"

/Users/imrisofer/miniconda3/envs/pymc-marketing-dev/lib/python3.12/site-packages/pymc_extras/model/marginal/graph_analysis.py:10: FutureWarning: `pytensor.graph.basic.io_toposort` was moved to `pytensor.graph.traversal.io_toposort`. Calling it from the old location will fail in a future release.
  from pytensor.graph.basic import io_toposort


In [3]:
folder_name = "/Users/imrisofer/projects/pymc-marketing/pymc_marketing/sandbox"
fname = f"{folder_name}/multidimensional_mmm_case_study.pm"
mmm = MMM.load(fname, check=False)

### Q: How did the contribution of each channel changed year after year?

In [ ]:
from pymc_marketing.mmm.plot_interactive import MMMPlotlyFactory

factory = MMMPlotlyFactory(summary=mmm.summary)
fig = factory.contributions(frequency="yearly", color="date", x="channel")
fig.show("notebook")
# after integration is complete this would be done with:
# mmm.plot_interactive.contributions(frequency="yearly", color="date", x="channel")

### Q: Within each year which channel perform better in each of the geos?

In [5]:
factory.contributions(frequency="yearly", color="channel", x="date")

### Q: looking over all the data, which channel performed better in each geo?

In [6]:
factory.contributions(frequency="all_time")

### Q: aggregating over geos, how did the contribution of each channel changed year after year?

In [7]:
from pymc_marketing.mmm.summary import MMMSummaryFactory

agg_data = mmm.data.aggregate_dims(
    dim="geo", values=["geo_a", "geo_b"], new_label="all_geos"
)
agg_summary = MMMSummaryFactory(agg_data, mmm)
agg_factory = MMMPlotlyFactory(summary=agg_summary)
agg_factory.contributions(frequency="yearly", color="channel", x="date")

### Q: aggregating over geos, how did the contribution of each channel changed quarter after quarter starting 2024?

In [8]:
from pymc_marketing.mmm.summary import MMMSummaryFactory

filtered_data = mmm.data.filter_dates(start_date="2024-01-01")
filtered_summary = MMMSummaryFactory(filtered_data, mmm)
filtered_factory = MMMPlotlyFactory(summary=filtered_summary)
filtered_factory.contributions(frequency="quarterly", color="channel", x="date")

### Q: show me the same, but without errorbars

In [9]:
filtered_factory.contributions(
    frequency="quarterly", color="channel", x="date", hdi_prob=None
)

In [10]:
factory.contributions(frequency="yearly", x="date", color="channel")

In [11]:
try:
    factory.contributions(component="control", frequency="yearly", x="date")
except ValueError as e:
    print("Got expected error:", e)

Got expected error: choose either x=`control` or color=`control`
